## Element-wise tensor operations

#### An element-wise operation is an operation between two tensors that operates on corresponding elements within the respective tensors. Two elements are said to be corresponding if the two elements occupy the same position within the tensor

In [2]:
import torch
import numpy as np

In [2]:
t1 = torch.tensor([[1,2],[3,4]])
t2 = torch.tensor([[5,6],[7,8]])

In [3]:
t1

tensor([[1, 2],
        [3, 4]])

In [4]:
t2

tensor([[5, 6],
        [7, 8]])

In [5]:
print(t1[0][0])
print(t2[0][0])
#corresponding elements 

tensor(1)
tensor(5)


### Two tensors must have the same shape in order to perform element-wise operations on them.

Lets look on basic element wise operations


In [6]:
t1 + t2 

tensor([[ 6,  8],
        [10, 12]])

In [7]:
t1  - t2 

tensor([[-4, -4],
        [-4, -4]])

In [8]:
t1 * t2 

tensor([[ 5, 12],
        [21, 32]])

In [9]:
t1 / t2 

tensor([[0, 0],
        [0, 0]])

In [10]:
t2 / t1 

tensor([[5, 3],
        [2, 2]])

In [11]:
t2 % t1 

tensor([[0, 0],
        [1, 0]])

In [12]:
t1 % t2 

tensor([[1, 2],
        [3, 4]])

In [13]:
t1 + 2 
# 2 is a scalar value i.e. is it is rank zero tensor while t1 is a rank 2 tensor but we still get correct value . HOW?


tensor([[3, 4],
        [5, 6]])

##### This is where broadcasting comes into play 
##### Broadcasting describes how tensors with different shapes are treated during element-wise operations.
### Broadcasting is the concept whose implementation allows us to add scalars to higher dimensional tensors.
Let's think about the t1 + 2 operation. Here, the scaler valued tensor is being broadcasted to the shape of t1, and then, the element-wise operation is carried out.
We can see what the broadcasted scalar value looks like using the broadcast_to() Numpy function:




In [14]:
np.broadcast_to(2, t1.shape)

array([[2, 2],
       [2, 2]])

In [15]:
# so t1 + 2 is infact 
t1 + torch.tensor(
    np.broadcast_to(2, t1.shape)
)

tensor([[3, 4],
        [5, 6]])

In [16]:
t3 = torch.tensor([[1,1],[1,1]])
t4 = torch.tensor([2,4])

In [17]:
t3 

tensor([[1, 1],
        [1, 1]])

In [18]:
t4

tensor([2, 4])

In [19]:
t3 + t4 
# The lower rank tensor t4 will be transformed via broadcasting to match the shape of the higher rank tensor t3
#and the element-wise operation will be performed as usual.

tensor([[3, 5],
        [3, 5]])

In [20]:
#at background the scene are 
t3 + torch.tensor(np.broadcast_to(t4,t3.shape))

tensor([[3, 5],
        [3, 5]])

### When do we actually use broadcasting? We often need to use broadcasting when we are preprocessing our data, and especially during normalization routines.



## Comparison operations are element-wise
##### For a given comparison operations between tensors, a new tensor of the same shape is returned with each element containing either a 0 or a 1. 0 for False and 1 for True 

In [21]:
t5 = torch.tensor([
    [0,5,0],
    [6,0,7],
    [0,8,0]
])

In [22]:
t5.eq(0)
#equal to 

tensor([[ True, False,  True],
        [False,  True, False],
        [ True, False,  True]])

In [23]:
t5.ge(0)
#greater than equal to

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [25]:
t5.gt(0)
#greater than 

tensor([[False,  True, False],
        [ True, False,  True],
        [False,  True, False]])

In [26]:
t5.le(0)
#less than equal to 

tensor([[ True, False,  True],
        [False,  True, False],
        [ True, False,  True]])

In [28]:
t5.lt(0)
#less than 

tensor([[False, False, False],
        [False, False, False],
        [False, False, False]])

In [31]:
# these all use broadcast to work . Example the last example would be like below in background
t5 < torch.tensor(
    np.broadcast_to(0, t5.shape)
    
)

tensor([[False, False, False],
        [False, False, False],
        [False, False, False]])

### Element-wise operations using functions

In [32]:
t5.abs() 
#absolute value 

tensor([[0, 5, 0],
        [6, 0, 7],
        [0, 8, 0]])

In [34]:
t5.neg()
#negation or multiply by -1 
#similarly sqrt function and many more 

tensor([[ 0, -5,  0],
        [-6,  0, -7],
        [ 0, -8,  0]])

# Broadcsting details 

## Example 1: Same rank, different shapes


### STEP 1 Based on the tensors’ original shapes, there may not be a way to reshape them to force them to be compatible, and if we can’t do that, then we can’t use broadcasting.

We compare the shapes of the two tensors, starting at their last dimensions and working backwards. Our goal is to determine whether each dimension between the two tensors’ shapes is compatible.

The dimensions are compatible when either:

1.) They’re equal to each other.
2.) One of them is 1.

### Step 2: Determine the shape of the resulting tensor
Comparing the shape of (1, 3) to (3, 1), we first calculate the max of the last dimension.

The max of 3 and 1 is 3. 3 will be the last dimension of the shape of the resulting tensor.

Moving on to the next dimension, again, the max of 1 and 3 is 3. So, 3 will be the next dimension of the shape of the resulting tensor.

We’ve now stepped through each dimension of the shapes of the original tensors. We can conclude that the resulting tensor will have shape (3, 3).

In [3]:
t6 = torch.tensor([[1, 2, 3]])
t6.shape

torch.Size([1, 3])

In [4]:
len(t6.shape)

2

In [7]:
t7 = torch.tensor([[4],
 [5],
 [6]])
t7.shape

torch.Size([3, 1])

In [40]:
len(t7.shape)

2

In [44]:
t6 + t7

tensor([[5, 6, 7],
        [6, 7, 8],
        [7, 8, 9]])

In [48]:
t6 + torch.tensor(np.broadcast_to(t7,t6.shape))
#this will not work as both have same rank ..this works only when ranks are different 
#correct way is described below 

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (3,1) and requested shape (1,3)

In [8]:
 torch.tensor(np.broadcast_to(t6,[3,3])) + torch.tensor(np.broadcast_to(t7,[3,3]))
    #we have to change both the tensor to 3* 3 tensor why and how 3* 3? 

tensor([[5, 6, 7],
        [6, 7, 8],
        [7, 8, 9]])

### Tensor 1 broadcast to shape (3,3) and Tensor 2 broadcast to shape (3,3) and now row wise opertion is done

## Broadcasting Example 2 : Different ranks


In [46]:
t8 = torch.tensor([[1, 2, 3],])
t8.shape

torch.Size([1, 3])

In [47]:
t8 + 2 

tensor([[3, 4, 5]])

In [50]:
t8 + torch.tensor(np.broadcast_to(2,t8.shape))

tensor([[3, 4, 5]])



##### When we’re in a situation where the ranks of the two tensors aren’t the same, like what we have here, then we simply substitute a one in for the missing dimensions of the lower-ranked tensor.
##### In our example, we substitute a one for both missing dimensions in the scalar's shape, making it now have shape (1,1)

now we check compatiblity and shape of output tensor . 
Tensor 2 broadcast to shape (1,3)

In [51]:
t9 = torch.tensor([[[1, 2, 3],
  [4, 5, 6]]])
t9

tensor([[[1, 2, 3],
         [4, 5, 6]]])

In [52]:
t10 = torch.tensor([[1, 1, 1],
 [2, 2, 2],
 [3, 3, 3]]
)
t10

tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])

In [53]:
t9.shape

torch.Size([1, 2, 3])

In [54]:
t10.shape

torch.Size([3, 3])

In [55]:
t9 + t10
# we can not add these because they can not be broadcasted to match the other , since the number of elements are 
#not the same 

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

### Number of elements of both tensor should be equal to broadcast 